In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU
from keras.optimizers import SGD

In [4]:
AMZN = yf.download('AMZN', 
                      start='2014-01-01', 
                      end='2019-12-31', 
                      progress=False)
all_data = AMZN[['Adj Close','Open', 'High', 'Low', 'Close', 'Volume']].round(2)

In [6]:
all_data.head()

,Adj Close,Open,High,Low,Close,Volume
Date,,,,,,
2014-01-02,19.90,19.94,19.97,19.70,19.90,42756000
2014-01-03,19.82,19.91,20.14,19.81,19.82,44204000
2014-01-06,19.68,19.79,19.85,19.42,19.68,63412000
2014-01-07,19.90,19.75,19.92,19.71,19.90,38320000
2014-01-08,20.10,19.92,20.15,19.80,20.10,46330000


In [7]:
def ts_train_test(all_data,time_steps,for_periods):
    # create training and test set
    ts_train = all_data[:'2018'].iloc[:,0:1].values
    ts_test  = all_data['2019':].iloc[:,0:1].values
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)

    # create training data of s samples and t time steps
    X_train = []
    y_train = []
    y_train_stacked = []
    for i in range(time_steps,ts_train_len-1): 
        X_train.append(ts_train[i-time_steps:i,0])
        y_train.append(ts_train[i:i+for_periods,0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshaping X_train for efficient modelling
    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

    inputs = pd.concat((all_data["Adj Close"][:'2018'], all_data["Adj Close"]['2019':]),axis=0).values
    inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
    inputs = inputs.reshape(-1,1)

    # Preparing X_test
    X_test = []
    for i in range(time_steps,ts_test_len+time_steps-for_periods):
        X_test.append(inputs[i-time_steps:i,0])
        
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    return X_train, y_train , X_test

In [8]:
X_train, y_train, X_test = ts_train_test(all_data,5,2)
X_train.shape

(1252, 5, 1)

In [9]:
model = Sequential()
model.add(GRU(units=40, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(GRU(units=30, activation='tanh'))
model.add(Dense(units=2))

model.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [10]:
model

In [12]:
X_train.shape

(1252, 5, 1)

In [13]:
y_train

array([[20.05, 19.88],
       [19.88, 19.55],
       [19.55, 19.88],
       ...,
       [73.54, 73.08],
       [73.08, 73.9 ],
       [73.9 , 75.1 ]])

In [14]:
model.fit(X_train,y_train,epochs=20, batch_size=60, verbose=1)

Epoch 1/20
21/21 [==============================] - 4s 5ms/step - loss: 878.5734
Epoch 2/20
21/21 [==============================] - 0s 4ms/step - loss: 608.4713
Epoch 3/20
21/21 [==============================] - 0s 4ms/step - loss: 603.8295
Epoch 4/20
21/21 [==============================] - 0s 4ms/step - loss: 569.6635
Epoch 5/20
21/21 [==============================] - 0s 4ms/step - loss: 567.8776
Epoch 6/20
21/21 [==============================] - 0s 5ms/step - loss: 579.6353
Epoch 7/20
21/21 [==============================] - 0s 4ms/step - loss: 571.4412
Epoch 8/20
21/21 [==============================] - 0s 5ms/step - loss: 576.7543
Epoch 9/20
21/21 [==============================] - 0s 4ms/step - loss: 575.7128
Epoch 10/20
21/21 [==============================] - 0s 4ms/step - loss: 575.7487
Epoch 11/20
21/21 [==============================] - 0s 4ms/step - loss: 575.4788
Epoch 12/20
21/21 [==============================] - 0s 4ms/step - loss: 586.8168
Epoch 13/20
21/21 [======

In [15]:
X_test.shape

(249, 5, 1)

In [16]:
predictions = model.predict(X_test)

8/8 [==============================] - 1s 3ms/step


In [17]:
X_test[0]

array([[67.2 ],
       [73.54],
       [73.08],
       [73.9 ],
       [75.1 ]])

In [ ]:
predictions